In [1]:
import nltk
#nltk.download()

In [2]:
import requests
import pandas as pd
import re, string, unicodedata
from pprint import pprint
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

#import contractions
#import inflect

In [3]:
MADRID_URL = "https://www.madridiario.es/sucesos"

In [4]:
def get_soup(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser") #parser para que no se queje
    return soup

soup = get_soup(MADRID_URL)

In [5]:
def get_headers(soup):
    news = []
    headers_container = soup.find_all("h2", {'class': 'titulo'})
    for header in headers_container:
        news.append(header.text)
        
    #print(news[:7])    
    return pd.DataFrame(data=news, columns=['Headers'])

dfnews_raw = get_headers(soup) #return dataframe
#print(type(dfnews_raw))

dfnews_raw.head(7)

,Headers
0,Un paracaidista cae a un tejado en el encuentr...
1,Un accidente entre un coche y una moto se sald...
2,Un hombre sufre varios traumatismos graves al ...
3,Desarticulado un grupo criminal que se dedicab...
4,Herido muy grave un hombre invidente y sordomu...
5,La sentencia del Supremo a 'La Manada' reaviva...
6,Localizado el argentino de 72 años desaparecid...


In [6]:
# Get rid of accents
def clean_accent(header_str):
    return header_str.replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u')
#df = df.replace({' ': ''}, regex=True)

dfnews_raw2 = pd.DataFrame(dfnews_raw['Headers'].apply(clean_accent))
dfnews_raw2.head(7)

,Headers
0,Un paracaidista cae a un tejado en el encuentr...
1,Un accidente entre un coche y una moto se sald...
2,Un hombre sufre varios traumatismos graves al ...
3,Desarticulado un grupo criminal que se dedicab...
4,Herido muy grave un hombre invidente y sordomu...
5,La sentencia del Supremo a 'La Manada' reaviva...
6,Localizado el argentino de 72 años desaparecid...


In [7]:
#Get rid of commas
def clean_commas(header_str):
    return header_str.replace(',','')

dfnews_raw3 = pd.DataFrame(dfnews_raw2['Headers'].apply(clean_accent))
dfnews_raw3.head(7)

,Headers
0,Un paracaidista cae a un tejado en el encuentr...
1,Un accidente entre un coche y una moto se sald...
2,Un hombre sufre varios traumatismos graves al ...
3,Desarticulado un grupo criminal que se dedicab...
4,Herido muy grave un hombre invidente y sordomu...
5,La sentencia del Supremo a 'La Manada' reaviva...
6,Localizado el argentino de 72 años desaparecid...


In [9]:
# Get each word / Tokenizar
def token(header_str):
    return nltk.word_tokenize(header_str)

token_dfnews = pd.DataFrame(dfnews_raw3['Headers'].apply(token))
token_dfnews.head(7)



<Text: Headers...>


In [19]:
stopWords = set(stopwords.words('spanish'))


AttributeError: 'function' object has no attribute 'words'

In [20]:
print(stopWords)

{'estas', 'todo', 'qué', 'habremos', 'sentida', 'tendría', 'nosotros', 'estéis', 'y', 'teníamos', 'sean', 'algunas', 'haya', 'estará', 'estuviesen', 'estados', 'al', 'esas', 'estemos', 'tendrás', 'tuviesen', 'hay', 'hubieras', 'tengas', 'sentidas', 'donde', 'poco', 'soy', 'no', 'estuvieses', 'quien', 'fueras', 'hubieseis', 'hubiésemos', 'mí', 'tu', 'todos', 'o', 'habíais', 'tuviera', 'habido', 'habidos', 'entre', 'lo', 'estuvieseis', 'las', 'nuestras', 'habiendo', 'estén', 'estaríais', 'ese', 'eran', 'sintiendo', 'estarán', 'esté', 'fueseis', 'eso', 'eras', 'hubiste', 'estadas', 'hubierais', 'habrán', 'estamos', 'hubiese', 'míos', 'una', 'estarías', 'sentido', 'fuimos', 'otras', 'sentid', 'estés', 'unos', 'tengo', 'nada', 'estoy', 'estos', 'tiene', 'cuando', 'tendrá', 'sea', 'su', 'erais', 'tuvieron', 'sí', 'hubiéramos', 'tuviste', 'era', 'estarás', 'la', 'habrías', 'estaban', 'estás', 'estabas', 'vosostras', 'hubisteis', 'ya', 'sois', 'fuiste', 'tuvierais', 'habías', 'habré', 'tenidas

In [34]:
def clean_stopwords(list_tokens):
    for token in list_tokens:
        for word in stopWords:
            if word == token:
                list_tokens.remove(word)
    return list_tokens


In [35]:
new = ['todo','es','un','caca','no','me','toques','los','cojones']


In [36]:
stopwords(new)

['es', 'caca', 'me', 'toques', 'cojones']

In [37]:
clean_dfnews = pd.DataFrame(token_dfnews['Headers'].apply(clean_stopwords))
clean_dfnews.head()

,Headers
0,"[Un, paracaidista, cae, un, tejado, el, encuen..."
1,"[Un, accidente, un, coche, una, moto, salda, u..."
2,"[Un, hombre, sufre, varios, traumatismos, grav..."
3,"[Desarticulado, grupo, criminal, se, dedicaba,..."
4,"[Herido, grave, hombre, invidente, sordomudo, ..."
